In [10]:
window_size_ma=20
threshold_param=100
long_probability_threshold=0.55
short_probability_threshold=0.55
fees = 0.0015 
stop_loss = 0.01
sl_trail = 0.05
entry1=0.7
entry2=0.55
entry3=0.15
entry4=0.05
# position_sizes = {
#     20: 0.02,  
#     50: 0.25,  
#     70: 0.85   
# }

In [11]:
import pandas as pd
import vectorbt as vbt
import numpy as np

data = pd.read_csv('btc_1h_test.csv')
column_names = ['datetime', 'open', 'high', 'low', 'close', 'volume']
data.columns = column_names

data['datetime'] = pd.to_datetime(data['datetime'])
data.set_index('datetime', inplace=True)


In [12]:
from joblib import load
from sklearn.preprocessing import StandardScaler
from ta.volatility import AverageTrueRange

loaded_clf = load('saved_rf_model.joblib')


data['MA'] = data['close'].rolling(window=window_size_ma).mean()

data['price_diff'] = data['close'].diff()
data['target'] = (data['price_diff'] > threshold_param).astype(int)

data['ATR'] = AverageTrueRange(data['high'], data['low'], data['close'], window=14).average_true_range()

for lag in range(1, 5):
    data[f'lag_{lag}_ATR'] = data['ATR'].shift(lag)

data.dropna(inplace=True)


X_new = data[['MA', 'ATR', 'lag_1_ATR', 'lag_2_ATR', 'lag_3_ATR', 'lag_4_ATR']]

scaler = StandardScaler()
X_new_scaled = scaler.fit_transform(X_new)


predictions = loaded_clf.predict(X_new_scaled)  

data['predictions'] = predictions

In [13]:
price_increase_prob = loaded_clf.predict_proba(X_new_scaled)[:, 1]  
price_decrease_prob = loaded_clf.predict_proba(X_new_scaled)[:, 0]  

In [14]:
data['entry_long'] = (
    (data['low'] > data['low'].shift(1)) &
    (data['close'] > data['high'].shift(1)) 
    & (price_increase_prob >= long_probability_threshold) 
).astype(int)

data['exit_long'] = (
    (data['close'] < data['low'].shift(1)) &
    (data['high'] < data['high'].shift(1))
).astype(int)

data['entry_short'] = (
    (data['high'] < data['high'].shift(1)) &
    (data['close'] < data['low'].shift(1)) 
    &(price_increase_prob >= short_probability_threshold)  
).astype(int)

data['exit_short'] = (
    (data['close'] > data['high'].shift(1)) &
    (data['low'] > data['low'].shift(1))
).astype(int)
data

,open,high,low,close,volume,MA,price_diff,target,ATR,lag_1_ATR,lag_2_ATR,lag_3_ATR,lag_4_ATR,predictions,entry_long,exit_long,entry_short,exit_short
datetime,,,,,,,,,,,,,,,,,,
2021-02-01 20:30:00,33535.51,33638.99,33160.00,33160.17,2721.186603,33533.1080,-372.64,0,578.710750,586.381577,595.604006,611.035083,591.114705,0,0,0,0,0
2021-02-01 21:30:00,33160.00,33603.91,33133.26,33460.83,2419.872780,33563.4630,300.66,1,570.992125,578.710750,586.381577,595.604006,611.035083,0,0,0,0,0
2021-02-01 22:30:00,33460.83,34100.00,33380.00,33978.31,3038.668685,33634.3420,517.48,1,581.635544,570.992125,578.710750,586.381577,595.604006,0,0,0,0,1
2021-02-01 23:30:00,33979.94,34032.93,33655.01,33731.52,1872.630656,33672.2130,-246.79,0,567.084434,581.635544,570.992125,578.710750,586.381577,0,0,0,0,0
2021-02-02 00:30:00,33739.52,34072.54,33700.30,33838.13,1802.347253,33709.4705,106.61,1,553.166975,567.084434,581.635544,570.992125,578.710750,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-31 01:30:00,37512.37,37740.00,37351.63,37684.45,1037.922560,37956.4560,172.08,1,299.172369,292.311013,285.210322,291.257270,294.957829,0,0,0,0,0
2022-01-31 02:30:00,37684.45,37762.60,37492.25,37729.00,874.060520,37946.3185,44.55,0,297.113629,299.172369,292.311013,285.210322,291.257270,0,0,0,0,0
2022-01-31 03:30:00,37729.00,37855.54,37625.41,37785.78,734.658550,37938.0175,56.78,0,292.329084,297.113629,299.172369,292.311013,285.210322,0,0,0,0,1


In [15]:
num_entry_long = data['entry_long'].sum()
num_exit_long = data['exit_long'].sum()
num_entry_short = data['entry_short'].sum()
num_exit_short = data['exit_short'].sum()
print(num_entry_long)
print(num_exit_long)
print(num_entry_short)
print(num_exit_short)

122
1614
138
1652


In [16]:
data

,open,high,low,close,volume,MA,price_diff,target,ATR,lag_1_ATR,lag_2_ATR,lag_3_ATR,lag_4_ATR,predictions,entry_long,exit_long,entry_short,exit_short
datetime,,,,,,,,,,,,,,,,,,
2021-02-01 20:30:00,33535.51,33638.99,33160.00,33160.17,2721.186603,33533.1080,-372.64,0,578.710750,586.381577,595.604006,611.035083,591.114705,0,0,0,0,0
2021-02-01 21:30:00,33160.00,33603.91,33133.26,33460.83,2419.872780,33563.4630,300.66,1,570.992125,578.710750,586.381577,595.604006,611.035083,0,0,0,0,0
2021-02-01 22:30:00,33460.83,34100.00,33380.00,33978.31,3038.668685,33634.3420,517.48,1,581.635544,570.992125,578.710750,586.381577,595.604006,0,0,0,0,1
2021-02-01 23:30:00,33979.94,34032.93,33655.01,33731.52,1872.630656,33672.2130,-246.79,0,567.084434,581.635544,570.992125,578.710750,586.381577,0,0,0,0,0
2021-02-02 00:30:00,33739.52,34072.54,33700.30,33838.13,1802.347253,33709.4705,106.61,1,553.166975,567.084434,581.635544,570.992125,578.710750,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-31 01:30:00,37512.37,37740.00,37351.63,37684.45,1037.922560,37956.4560,172.08,1,299.172369,292.311013,285.210322,291.257270,294.957829,0,0,0,0,0
2022-01-31 02:30:00,37684.45,37762.60,37492.25,37729.00,874.060520,37946.3185,44.55,0,297.113629,299.172369,292.311013,285.210322,291.257270,0,0,0,0,0
2022-01-31 03:30:00,37729.00,37855.54,37625.41,37785.78,734.658550,37938.0175,56.78,0,292.329084,297.113629,299.172369,292.311013,285.210322,0,0,0,0,1


In [17]:
data.loc[data['entry_long'] == 1, 'stop_loss'] = data['low'].shift(1)
data.loc[data['entry_short'] == 1, 'stop_loss'] = data['high'].shift(1)
data.dropna(inplace=True)

In [23]:
print("Length of price_increase_prob:", len(price_increase_prob))
print("Length of data:", len(data))


Length of price_increase_prob: 8709
Length of data: 260


In [18]:
data['position_size'] = np.where(
    (data['predictions'] == 1) & (price_increase_prob > 0.85),
    entry1,  
    np.where(
        (data['predictions'] == 1) & (price_increase_prob > 0.75),
        entry2,  
        np.where(
            (data['predictions'] == 1) & (price_increase_prob > 0.65),
            entry3,  
            np.where(
                (data['predictions'] == 1) & (price_increase_prob > 0.55),
                entry4,  
                0.0   
            )
        )
    )
)

value_counts = data['position_size'].value_counts()
print("Unique values:", value_counts.index.tolist())
print("Value counts:\n", value_counts)


ValueError: operands could not be broadcast together with shapes (260,) (8709,) 

In [21]:

portfolio = vbt.Portfolio.from_signals(freq="1H",
    close=data['close'],
    entries=data['entry_long'],
    exits=data['exit_long'],
    short_entries=data['entry_short'],
    short_exits=data['exit_short'],
    sl_stop=data['stop_loss'],  
    fees=fees,
    init_cash=100000
)

portfolio_stats = portfolio.stats()
portfolio_stats

Start                               2021-02-02 14:30:00
End                                 2022-01-26 23:30:00
Period                                 10 days 20:00:00
Start Value                                    100000.0
End Value                                  41712.166602
Total Return [%]                             -58.287833
Benchmark Return [%]                           7.435068
Max Gross Exposure [%]                            100.0
Total Fees Paid                            22158.453867
Max Drawdown [%]                              60.967393
Max Drawdown Duration                  10 days 19:00:00
Total Trades                                        120
Total Closed Trades                                 119
Total Open Trades                                     1
Open Trade PnL                                -62.56825
Win Rate [%]                                  35.294118
Best Trade [%]                                28.851587
Worst Trade [%]                              -28

In [22]:
portfolio.plot().show()